In [1]:
%load_ext autoreload
%autoreload 2
import glob
import numpy as np
import pandas as pd
import numpy as np
from pathlib import Path
import json
from tqdm.notebook import tqdm, trange
from sglm.models import sglm
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf
from collections import defaultdict
import numpy as np
import scipy.stats as scs
import itertools

In [2]:
src = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_9010')
# src = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_7030')

def create_basis(file_dir):
    files = [Path(_).parts[-1] for _ in glob.glob(file_dir) if 'SIGNALS' in _]

    basis = pd.DataFrame(files, columns=['signal_file'])
    basis['table_file'] = basis['signal_file'].str.replace('GLM_SIGNALS_', 'GLM_TABLE_')
    basis['figname'] = 'fig'
    basis['groupid'] = 'g'
    basis['mouseid'] = basis['signal_file'].str.replace('GLM_SIGNALS_', '').str.slice(0,8)
    basis['rename_dict'] = "{'Ch1': 'GCAMP'}"
    basis['signal_file_out'] = basis['signal_file'].str.replace('GLM_SIGNALS_', 'GLM_SIGNALS_INTERIM_')
    basis['table_file_out'] = basis['table_file'].str.replace('GLM_TABLE_', 'GLM_TABLE_INTERIM_')
    return basis

create_basis(str((src / '*').resolve())).to_csv(str((src / '_basis.csv').resolve()))

In [3]:
# sorted([Path(_).parts[-1].replace('SIGNALS','SIGNALS_INTERIM').replace('TABLE', 'TABLE_INTERIM') for _ in glob.glob('/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/lynne-sanity/*')])


In [4]:
num_sft = (-20, 20)
# default_source_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\raw')
# default_output_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\interim')

# default_source_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/raw')
# default_output_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/interim_check')

# default_source_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/mike-GLM_8020-raw')
# default_output_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/mike-GLM_8020-interim')


# default_source_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/lynne-sanity')
# default_output_folder = Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/lynne-sanity-interim')

default_source_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_7030')
default_output_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_7030-interim')

# default_source_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_8020')
# default_output_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_8020-interim')

# default_source_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_9010')
# default_output_folder = Path(r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_9010-interim')

output_json_name = Path('preproc_params.json')
basis_name = Path('_basis.csv')

In [5]:
y_col_lst_all = ['GCAMP', 'Ch1']

In [6]:
params = {'sft': num_sft, 'y_cols_all': y_col_lst_all}
default_output_folder.mkdir(parents=True, exist_ok=True)
with open(str((default_output_folder / output_json_name).resolve()), 'w') as json_file:
    json.dump(params, json_file)

In [7]:
from tqdm.notebook import tqdm

In [8]:
basis = pd.read_csv(str((default_source_folder / basis_name).resolve()))
basis.to_csv(str((default_output_folder / basis_name).resolve()))
source_folder = default_source_folder

interim_preproc_errors = defaultdict(list)

for inx, conversion in tqdm(basis.iterrows(), total=len(basis)):
#     print(conversion)
#     if 'source_folder' in conversion.index:
#         source_folder = Path(conversion['source_folder'])
#     else:
#         source_folder = default_source_folder
    
    figname = Path(conversion['figname'])
    groupid = Path(conversion['groupid'])
    signal_file = Path(conversion['signal_file'])
    table_file = Path(conversion['table_file'])
    rename_dict = eval(conversion['rename_dict'])
    signal_file_out = Path(conversion['signal_file_out'])
    table_file_out = Path(conversion['table_file_out'])
    
    
    signal_path = str((source_folder / signal_file).resolve())
    table_path = str((source_folder / table_file).resolve())
    
    out_folder = default_output_folder / figname / groupid
    out_folder.mkdir(parents=True, exist_ok=True)
    
    signal_path_out = default_output_folder / figname / groupid / signal_file_out
    table_path_out = default_output_folder / figname / groupid / table_file_out
    
    if signal_path_out.exists():
        raise OSError(f'Signal Path {signal_path_out} already exists!')
    if table_path_out.exists():
        raise OSError(f'Table Path {table_path_out} already exists!')
    
    signal_df = pd.read_csv(signal_path)
    table_df = pd.read_csv(table_path)
    
    brk = False
    # Check for multiple copies of the same sideIn
    for multiple_copy_check in ['photometryCenterInIndex', 'photometryCenterOutIndex',
                                'photometrySideInIndex', 'photometrySideOutIndex',]:
        eq = table_df[multiple_copy_check].dropna()
        eq = eq[eq != 0]
        if len(eq) != eq.nunique():
            interim_preproc_errors[(figname, groupid)].append({'in_file_errors': signal_file, 'out_file_errors': signal_file_out})

            print(len(eq))
            print(eq.nunique())
            display(eq)
            print(f'Error: Multiple {multiple_copy_check} with the same timestamp detected for {signal_path}. Continuing...')
            brk = True
            break
        elif (figname, groupid) not in interim_preproc_errors:
            interim_preproc_errors[(figname, groupid)] = []
    
    if brk:
        continue
    elif "GLM_SIGNALS_s stflp26_01252021" in str(signal_file.resolve()):
        print(f'Excluding {signal_file} and continuing...')
        interim_preproc_errors[(figname, groupid)].append({'in_file_errors': signal_file, 'out_file_errors': signal_file_out})
        continue
    
    
    print(signal_path)
    signal_df, table_df = gsd.generate_signal_df(signal_path,
                                            table_path,
                                            trial_bounds_before_center_in = num_sft[0],
                                            trial_bounds_after_side_out = num_sft[1],
                                            )
    
    assert np.all(table_df['choseLeft'] != table_df['choseRight'])
    
    signal_df = signal_df[signal_df['nTrial'] > 0].fillna(0)
    
    
    # Rename Columns
    signal_df = bf.rename_consistent_columns(signal_df)
    signal_df = signal_df.rename(rename_dict, axis=1)

    for y_col in y_col_lst_all:
        if y_col not in signal_df.columns:
            signal_df[y_col] = np.nan
            continue
    
    ## Set Full Trial Reward Flags
    signal_df['r_trial'] = (signal_df.groupby('nTrial')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
    signal_df['nr_trial'] = (signal_df.groupby('nTrial')['photometrySideInIndexnr'].transform(np.sum) > 0) * 1.0
    
    signal_df['nr_trial'] = (signal_df['nr_trial'] - signal_df['r_trial'] > 0) * 1.0
    
#     signal_df['rnr_err_trial'] = (signal_df['r_trial'] + signal_df['nr_trial'] != 1.0) * 1.0

    ## Define Side Rewarded / Unrewarded Flags
    signal_df = bf.set_port_entry_exit_rewarded_unrewarded_indicators(signal_df)
    
#     display(list(signal_df.columns))
    
    ## Define Side Agnostic Events
    signal_df = bf.define_side_agnostic_events(signal_df, agnostic_definitions={'spn':['rpn','lpn'],
                                                          'spx':['rpx','lpx'],
                                                          'spnr':['rpnr','lpnr'],
                                                          'spnnr':['rpnnr','lpnnr'],
                                                          'spxr':['rpxr','lpxr'],
                                                          'spxnr':['rpxnr','lpxnr'],
#                                                           'sl':['rl','ll'],
                                                         })
    
    # print('Percent of Data in ITI:', (df['nTrial'] == df['nEndTrial']).mean())

    signal_df['spnrOff'] = ((signal_df['spnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    signal_df['spxrOff'] = ((signal_df['spxr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)
    spnnrOff_a = ((signal_df['spnnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    spxnrOff_a = ((signal_df['spxnr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)

    # If we have something listed as a rewarded "off" side entry labeled in the table as a side exit... it means it was a fast "out-in".
    # The latter "in" should be considered an unrewarded side port "off" entry.
    dualism_exen = ((signal_df['spnrOff'] == 1)&(signal_df['photometrySideOutIndex'] == 1)).astype(int)

    # Unrewarded side port entries should be the combination of those simply identified by checking spnnr & the table labels +
    # the dualism defined immediately prior. Then those dualism examples should be remoed from the "off" rewarded entries.
    signal_df['spnnrOff'] = spnnrOff_a + dualism_exen
    signal_df['spnrOff'] = signal_df['spnrOff'] - dualism_exen
    signal_df['spxnrOff'] = spxnrOff_a
    
#     signal_df['slOff'] = signal_df['sl'] * signal_df['nr_trial']
#     signal_df['slOn'] = signal_df['sl'] - signal_df['slOff']
    
    signal_df['cpnOff'] = ((signal_df['cpn'] == 1)&(signal_df['photometryCenterInIndex'] != 1)).astype(int)
    signal_df['cpxOff'] = ((signal_df['cpx'] == 1)&(signal_df['photometryCenterOutIndex'] != 1)).astype(int)
    
    for resp in ['GCAMP']:
#     for resp in y_col_lst_all:
        print(f'{resp} Stats:')
        display(np.round(signal_df.groupby('wi_trial_keep')[resp].agg([np.mean, np.var, scs.skew, scs.kurtosis]),3))
    
    if signal_path_out:
        signal_df.to_csv(signal_path_out, index_label='index')
    if table_path_out:
        table_df.to_csv(table_path_out, index_label='index')
    
    assert np.max(signal_df['photometrySideInIndexr'] + signal_df['photometrySideInIndexnr']) == 1.0
    

for figname, groupid in tqdm(interim_preproc_errors, total=len(interim_preproc_errors)):
    interim_preproc_errors_path = default_output_folder / figname / groupid / 'interim_preproc_errors.csv'

    if len(interim_preproc_errors[(figname, groupid)]) != 0:
        interim_preproc_errors_df = pd.DataFrame(interim_preproc_errors[(figname, groupid)])
    else:
        interim_preproc_errors_df = pd.DataFrame(columns=['in_file_errors', 'out_file_errors'])

    interim_preproc_errors_df.to_csv(str(interim_preproc_errors_path.resolve()))


  0%|          | 0/49 [00:00<?, ?it/s]

C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_7030\GLM_SIGNALS_sstflp15_08152019.txt
GCAMP Stats:


,mean,var,skew,kurtosis
wi_trial_keep,,,,
False,-0.123,0.720,0.862,1.697
True,-0.002,1.077,0.987,1.407


C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_7030\GLM_SIGNALS_sstflp15_08202019.txt
GCAMP Stats:


,mean,var,skew,kurtosis
wi_trial_keep,,,,
False,-0.237,0.621,0.924,1.798
True,0.029,1.072,0.853,0.903


...

C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_7030\GLM_SIGNALS_sstflp26_05062021.txt
GCAMP Stats:


,mean,var,skew,kurtosis
wi_trial_keep,,,,
False,0.037,0.837,0.570,0.234
True,-0.009,0.964,0.674,0.734


C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\mike-GLM_7030\GLM_SIGNALS_sstflp26_05102021.txt
GCAMP Stats:


,mean,var,skew,kurtosis
wi_trial_keep,,,,
False,-0.061,0.831,0.653,0.826
True,0.020,0.970,0.681,0.716


  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
interim_preproc_errors_df

,in_file_errors,out_file_errors
0,GLM_SIGNALS_sstflp17_01092020.txt,GLM_SIGNALS_INTERIM_sstflp17_01092020.txt


In [10]:
interim_preproc_errors

defaultdict(list,
            {(WindowsPath('fig'),
              WindowsPath('g')): [{'in_file_errors': WindowsPath('GLM_SIGNALS_sstflp17_01092020.txt'),
               'out_file_errors': WindowsPath('GLM_SIGNALS_INTERIM_sstflp17_01092020.txt')}]})

In [11]:
# signal_df[['photometryCenterInIndexLt', 'photometryCenterInIndexRt']].iloc[:2000].plot()

In [12]:

# group_1_mice = ['WT63', 'WT64', 'WT65']
# # group_1_sess = ['11082021', '11102021', '11122021', '11182021']
# group_1_sess = ['11082021', '11102021', '11122021', '11162021', '11182021']
# group_1_combo = [('Figure_6', 'g1', _[0], f'GLM_SIGNALS_{_[0]}_{_[1]}.txt', f'GLM_TABLE_{_[0]}_{_[1]}.txt', {'Ch1': 'gDA', 'Ch5': 'gACH'}) for _ in list(itertools.product(group_1_mice, group_1_sess))]

# group_2_mice = ['WT66', 'WT67', 'WT68', 'WT69']
# group_2_sess = ['12132021', '12152021', '12172021', '12192021']
# group_2_combo = [('Figure_6', 'g2', _[0], f'GLM_SIGNALS_{_[0]}_{_[1]}.txt', f'GLM_TABLE_{_[0]}_{_[1]}.txt', {'Ch1': 'gDA'}) for _ in list(itertools.product(group_2_mice, group_2_sess))]

# group_3_mice = ['WT58', 'WT60', 'WT61']
# group_3_sess = ['10042021', '10062021', '10082021', '10112021', '10132021', '10152021']
# group_3_combo = [('Figure_6', 'g3', _[0], f'GLM_SIGNALS_{_[0]}_{_[1]}.txt', f'GLM_TABLE_{_[0]}_{_[1]}.txt', {'Ch1': 'gACH'}) for _ in list(itertools.product(group_3_mice, group_3_sess))]

# group_4_mice = ['WT53', 'WT54', 'WT55', 'WT56']
# group_4_sess = ['09012021', '09032021', '09062021']
# group_4_combo = [('Figure_6', 'g4', _[0], f'GLM_SIGNALS_{_[0]}_{_[1]}.txt', f'GLM_TABLE_{_[0]}_{_[1]}.txt', {'Ch1': 'gACH'}) for _ in list(itertools.product(group_4_mice, group_4_sess))]

# group_5_mice = ['WT57', 'WT59']
# group_5_sess = ['10042021', '10062021', '10082021', '10112021', '10132021', '10152021']
# group_5_combo = [('Figure_6', 'g5', _[0], f'GLM_SIGNALS_{_[0]}_{_[1]}.txt', f'GLM_TABLE_{_[0]}_{_[1]}.txt', {'Ch5': 'gACH'}) for _ in list(itertools.product(group_5_mice, group_5_sess))]

# group_6_mice = []#['WT62']
# group_6_sess = []#['11082021', '11102021', '11122021', '11182021']
# group_6_combo = [('Figure_6', 'g6', _[0], f'GLM_SIGNALS_{_[0]}_{_[1]}.txt', f'GLM_TABLE_{_[0]}_{_[1]}.txt', {'Ch1': 'gDA'}) for _ in list(itertools.product(group_6_mice, group_6_sess))]


# # channel_definitions = {}
# # # channel_definitions = {(file_combo,): {'Ch1': 'gACH', 'Ch2': 'rDA'} for file_combo in group_1_combo}
# # channel_definitions.update({(file_combo, 'Figure_1', 'g1'): {'Ch1': 'gDA', 'Ch5': 'gACH'} for file_combo in group_1_combo})
# # channel_definitions.update({(file_combo, 'Figure_1', 'g2'): {'Ch1': 'gDA'} for file_combo in group_2_combo})
# # channel_definitions.update({(file_combo, 'Figure_1', 'g3'): {'Ch1': 'gACH'} for file_combo in group_3_combo})
# # channel_definitions.update({(file_combo, 'Figure_1', 'g4'): {'Ch1': 'gACH'} for file_combo in group_4_combo})
# # channel_definitions.update({(file_combo, 'Figure_1', 'g5'): {'Ch5': 'gACH'} for file_combo in group_5_combo})
# # channel_definitions.update({(file_combo, 'Figure_1', 'g6'): {'Ch1': 'gDA'} for file_combo in group_6_combo})


# full_combo = []

# full_combo += group_1_combo
# full_combo += group_2_combo
# full_combo += group_3_combo
# full_combo += group_4_combo
# full_combo += group_5_combo
# full_combo += group_6_combo

# for fig, grp, mouse, sig, tbl, defi in full_combo:
#     if (Path(r'/Users/josh/Documents/github_repos/sabatinilab-glm/sglm/data/lynne-sanity') / sig).exists():
# #         print(f'{fig}, {grp}, {mouse}, {sig}, {tbl}, {defi}')
#         sig_int = sig.replace("SIGNALS", "SIGNALS_INTERIM")
#         tbl_int = tbl.replace("TABLE", "TABLE_INTERIM")
#         print(f'{sig_int}, {tbl_int}')
#     else:
#         print()

In [13]:
# signal_df[['photometrySideInIndex', 'photometrySideInIndexr', 'photometrySideInIndexnr',
#            'r_trial', 'nr_trial', 'rnr_err_trial', 'nTrial', 'nEndTrial']].loc[signal_df['rnr_err_trial'] == 1.0]


In [14]:
# basis = pd.read_csv(str((default_source_folder / f'_basis.csv').resolve()))
# source_folder = default_source_folder

# for inx, conversion in basis.iterrows():
#     figname = Path(conversion['figname'])
#     groupid = Path(conversion['groupid'])
#     signal_file = Path(conversion['signal_file'])
#     table_file = Path(conversion['table_file'])
#     rename_dict = eval(conversion['rename_dict'])
#     signal_file_out = Path(conversion['signal_file_out'])
#     table_file_out = Path(conversion['table_file_out'])
    
    
#     signal_path = str((source_folder / signal_file).resolve())
#     table_path = str((source_folder / table_file).resolve())
    
    
#     signal_path_out = default_output_folder / figname / groupid / signal_file_out
#     table_path_out = default_output_folder / figname / groupid / table_file_out
        
#     signal_df_out = pd.read_csv(signal_path_out)#.drop('rnr_err_trial',axis=1)
#     table_df_out = pd.read_csv(table_path_out)
#     old_signal_df = pd.read_csv(str(Path((r'C:\Users\Josh\Documents\GitHub\sabatinilab-glm\sglm\data\interim-old') / signal_file_out).resolve()))
    
#     assert np.all(old_signal_df.fillna(-1) == signal_df_out.fillna(-1))
    